# ATP dynamics in realistic mitochondridal geometries

Here, we implement the model presented in [Garcia et al, Scientific Reports](https://www.nature.com/articles/s41598-019-48028-0), which describes production of ATP in mitochondria.

The geometry in this model is divided into 4 domains - two volumes and two surfaces:
- outer mitochondrial membrane (OM)
- intermembrane space (IMS)
- Inner membrane (IM)
- Matrix (MAT) (volume inside the mitochondrion)

In [ ]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib
import logging
import gmsh  # must be imported before pyvista if dolfin is imported first

from smart import config, mesh, model, mesh_tools, visualization
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

First, we define the various units for the inputs and define universal constants.

In [ ]:
# Aliases - base units
mM = unit.mM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = mM * um / sec
vol_unit = mM
surf_unit = molecule / um**2

F = 9.649e4 # Faraday's constant (C/mol)
R = 8.315 # gas constant (J/mol-K)

## Model generation

We first define compartments and the add them to a compartment container. Note that we can specify nonadjacency for surfaces in the model, which is not required, but can speed up the solution process.

In [ ]:
IMS = Compartment("IMS", 3, um, 1)
OM = Compartment("OM", 2, um, 10)
Matrix = Compartment("Matrix", 3, um, 2)
IM = Compartment("IM", 2, um, 12)
OM.specify_nonadjacency(['IM', 'Matrix'])
IM.specify_nonadjacency(['OM'])

cc = CompartmentContainer()
cc.add([IMS, OM, Matrix, IM])

Define all model species and and place in species container.

In [ ]:
# ATP synthase states
E_tot = Parameter("E_tot", 267/1.545, surf_unit)
E_Mat = Species("E_Mat", 0.0, surf_unit, 0.0, D_unit, "IM")
E_IMS = Species("E_IMS", 1.0*E_tot.value, surf_unit, 0.0, D_unit, "IM")
E_Mat_H3Star = Species("E_Mat_H3Star", 0.0, surf_unit, 0.0, D_unit, "IM")
E_Mat_H3S = Species("E_Mat_H3S", 0.0, surf_unit, 0.0, D_unit, "IM")
E_Mat_H3 = Species("E_Mat_H3", 0.0, surf_unit, 0.0, D_unit, "IM")
# ANT states
L_tot = Parameter("L_tot", 16471/1.545, surf_unit)
L = Species("L", 1.0*L_tot.value, surf_unit, 0.0, D_unit, "IM")
TL = Species("TL", 0.0, surf_unit, 0.0, D_unit, "IM")
LT = Species("LT", 0.0, surf_unit, 0.0, D_unit, "IM")
DL = Species("DL", 0.0, surf_unit, 0.0, D_unit, "IM")
LD = Species("LD", 0.0, surf_unit, 0.0, D_unit, "IM")
TLD = Species("TLD", 0.0, surf_unit, 0.0, D_unit, "IM")
DLD = Species("DLD", 0.0, surf_unit, 0.0, D_unit, "IM")
DLDPrime = Species("DLDPrime", 0.0, surf_unit, 0.0, D_unit, "IM")
TLT = Species("TLT", 0.0, surf_unit, 0.0, D_unit, "IM")
TLTPrime = Species("TLTPrime", 0.0, surf_unit, 0.0, D_unit, "IM")
# ATP/ADP in matrix and IMS
D_Mat = Species("D_Mat", 0.8*2.0, vol_unit, 15.0, D_unit, "Mat")
T_Mat = Species("T_Mat", 13.0, vol_unit, 15.0, D_unit, "Mat")
T_IMS = Species("T_IMS", 6.5, vol_unit, 15.0, D_unit, "IMS")

Define mitochondrial conditions (temperature, voltage, pH)

In [ ]:
pH_Mat = 7.6 # matrix pH
pH_c = 7.2 # cristae pH
T = 310 # body temperature (K)
dPsi = 180 # membrane voltage (mV)
dPsiB_IMS = -50 # phase boundary potential bulk IMS -> IM (mV)
dPsiB_Mat = 0.0 # phase boundary potential from IM -> Mat (mV)
dPsi_m = dPsi - dPsiB_IMS - dPsiB_Mat

Now, we define parameters and reactions for each portion of the model. First, we define the ATP synthase dynamics:

In [ ]:
# E1: Movement of proton binding site in ATP synthase
k_16 = Parameter("k_16", 100.0*np.exp(3*F*dPsi_m/(2*R*T)), 1/sec) # movement of proton binding site Mat->IMS side
k_61 = Parameter("k_61", 4.98e7*np.exp(3*F*dPsi_m/(2*R*T)), 1/sec) # movement of proton binding site IMS->Mat side
E1 = Reaction("E1", ["E_Mat"], ["E_IMS"], {"on": "k_16", "off": "k_61"})

# E2: bind/release of 3 protons in IMS
k_65 = Parameter("k_65", 3969, 1/sec) # binding rate of protons in IMS
k_56 = Parameter("k_56", 2.75e5*np.exp(3*F*dPsiB_IMS/(R*T)), 1/sec) # release of protons in IMS
E2 = Reaction("E2", ["E_IMS"], [],
              param_map={"E_tot":"E_tot", "k_56":"k_56", "k_65":"k_65"}
              species_map={"E_Mat":"E_Mat", "E_IMS":"E_IMS", "E_Mat_H3Star":"E_Mat_H3Star", "E_Mat_H3S":"E_Mat_H3S", "E_Mat_H3":"E_Mat_H3"}
              eqn_f_str="k_65*E_IMS - k_56*(E_tot-E_Mat-E_IMS-E_Mat_H3Star-E_Mat_H3S-E_Mat_H3)")

# E3: movement of 3 protons from IMS to Matrix
k_54 = Parameter("k_54", 100.0, 1/sec)
k_45 = Parameter("k_45", 100.0, 1/sec)
E3 = Reaction("E3", [], ["E_Mat_H3Star"],
              param_map={"E_tot":"E_tot", "k_54":"k_54", "k_45":"k_45"}
              species_map={"E_Mat":"E_Mat", "E_IMS":"E_IMS", "E_Mat_H3Star":"E_Mat_H3Star", "E_Mat_H3S":"E_Mat_H3S", "E_Mat_H3":"E_Mat_H3"}
              eqn_f_str="k_54*(E_tot-E_Mat-E_IMS-E_Mat_H3Star-E_Mat_H3S-E_Mat_H3) - k_45*E_Mat_H3Star")

# E4: movement of 3 protons from IMS to matrix without producing ATP
k_52 = Parameter("k_54", 1e-20, 1/sec)
k_25 = Parameter("k_45", 5.85e-30, 1/sec)
E4 = Reaction("E4", [], ["E_Mat_H3"],
              param_map={"E_tot":"E_tot", "k_52":"k_52", "k_25":"k_25"}
              species_map={"E_Mat":"E_Mat", "E_IMS":"E_IMS", "E_Mat_H3Star":"E_Mat_H3Star", "E_Mat_H3S":"E_Mat_H3S", "E_Mat_H3":"E_Mat_H3"}
              eqn_f_str="k_52*(E_tot-E_Mat-E_IMS-E_Mat_H3Star-E_Mat_H3S-E_Mat_H3) - k_25*E_Mat_H3")

# E5: binding/unbinding of ADP-P to ATP synthase

# E6: ATP production

# E7: detachment of protons in the matrix
k_21 = Parameter("k_16", 40.0*np.exp(3*F*dPsiB_Mat/(2*R*T)), 1/sec) # unbinding rate of protons in matrix
k_12 = Parameter("k_61", 25.0, 1/sec) # binding rate of protons in matrix
E1 = Reaction("E1", ["E_Mat_H3"], ["E_Mat"], {"on": "k_21", "off": "k_12"})

## Create and load in mesh

Here, we consider an "ellipsoid-in-an-ellipsoid" geometry. The inner ellipsoid represents the ER and the volume between the ER boundary and the boundary of the outer ellipsoid represents the cytosol.

In [ ]:
# curRadius = 0.25  # dendritic spine radius
# domain, facet_markers, cell_markers = mesh_tools.create_ellipsoids((1.25*curRadius, 0.8*curRadius, curRadius),
#                                                                    (1.25*curRadius/2, 0.8 *
#                                                                     curRadius/2, curRadius/2),
#                                                                    hEdge=0.01)
cur_dir = pathlib.Path.cwd()
parent_dir = cur_dir.parent
mito_mesh = d.Mesh(f"{str(parent_dir)}/meshes/mito1_mesh.xml")
cell_markers = d.MeshFunction("size_t", mito_mesh, 3, mito_mesh.domains())
facet_markers_orig = d.MeshFunction("size_t", mito_mesh, 2, mito_mesh.domains())
facet_markers = d.MeshFunction("size_t", mito_mesh, 2, mito_mesh.domains())
facet_array = facet_markers.array()[:]
for i in range(len(facet_array)):
    if facet_array[i] == 11: # this indicates cristae
        facet_array[i] = 12 # consider cristae the same as inner membrane for now
    elif facet_array[i] == 0:
        facet_array[i] = 10
    elif facet_array[i] > 1e9:
        facet_array[i] = 0
# facet_markers.array()[np.where(facet_markers.array() > 1e9)[0]] = 0 # set unassigned facets to 0

# Write mesh and meshfunctions to file
mesh_folder = pathlib.Path("mesh")
mesh_folder.mkdir(exist_ok=True)
mesh_path = mesh_folder / "mito1.h5"
mesh_tools.write_mesh(
    mito_mesh, facet_markers, cell_markers, filename=mesh_path
)
parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_path),
    mesh_filetype="hdf5",
    name="parent_mesh",
)
visualization.plot_dolfin_mesh(mito_mesh, cell_markers, facet_markers, clip_logic=False)

Initialize model and solver.

In [ ]:
config_cur = config.Config()
config_cur.flags.update({"allow_unused_components": True})
model_cur = model.Model(pc, sc, cc, rc, config_cur, parent_mesh)
config_cur.solver.update(
    {
        "final_t": 0.1,
        "initial_dt": 0.001,
        "time_precision": 6,
    }
)
model_cur.initialize()

Initialize XDMF files for saving results, save model information to .pkl file, then solve the system until `model_cur.t > model_cur.final_t`

In [ ]:
# Write initial condition(s) to file
results = dict()
result_folder = pathlib.Path(f"results")
result_folder.mkdir(exist_ok=True)
for species_name, species in model_cur.sc.items:
    results[species_name] = d.XDMFFile(
        model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
model_cur.to_pickle("model_cur.pkl")

# Set loglevel to warning in order not to pollute notebook output
logger.setLevel(logging.WARNING)

concVec = np.array([.05])
cytoMesh = model_cur.cc['Cyto'].dolfin_mesh
# integrateDomain = d.MeshFunction("size_t", cytoMesh, 3, 0)
# RTarget = (curRadius + curRadius/2) / 2
# for c in d.cells(cytoMesh):
#     RCur = np.sqrt(c.midpoint().x()**2 + c.midpoint().y()**2 + c.midpoint().z()**2)
#     integrateDomain[c] = 1 if (RCur > RTarget-.1*curRadius and RCur <
#                                 RTarget + .1*curRadius) else 0
dx = d.Measure("dx", domain=cytoMesh)
volume = d.assemble(1.0*dx)
# Solve
displayed = False
while True:
    # Solve the system
    model_cur.monolithic_solve()
    # Save results for post processing
    for species_name, species in model_cur.sc.items:
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
    # save mean value at r = (curRadius + curRadius/2)/2 (for comparison to Cugno graph below)
    int_val = d.assemble(model_cur.sc['Ca'].u['u']*dx)
    curConc = np.array([int_val / volume])
    concVec = np.concatenate((concVec, curConc))
    np.savetxt(result_folder / f"tvec.txt", np.array(model_cur.tvec).astype(np.float32))
    # if model_cur.t > .025 and not displayed:  # display first time after .025 s
    #     visualization.plot(model_cur.sc['Ca'].u['u'])
    #     displayed = True
    # End if we've passed the final time
    if model_cur.t >= model_cur.final_t:
        break

Plot concentration over time.

In [ ]:
plt.plot(model_cur.tvec, concVec)
plt.xlabel("Time (s)")
plt.ylabel("Calcium concentration (μM)")
plt.title("SMART simulation")